# [CS303]Practice 6 - BTS & Practice 7 - Improving BTS
In Practice 6 and 7, we will apply the BTS algorithm and the Improving BTS to solve the N-Queens problem.

In the following code, only two of the function usages are essential. One is `is_valid(self, state)`, which checks if the current state is legal; the other is `is_satisfy(self, state)`, which is to checks if the current board meets the win condition. 
The type of state is [], which stores the tuples(a, b), representing the positions of the queens in it.

In test block, for `Solver`,  `n` indicates the size of the test while `method` indicates which bts function will be used(`bts` or `improving_bts`); for method `solve`, `render` indicates whether to use a graphical interface representation.

## Practice 6 - BTS
You are required to implement complete the function `bts()`.
You can only use iterative way, not recursive. Using recursion will incur a **20% penalty**. You may add any function you want. 
**Remember to test the case where N=6**.

📌 **DDL: Nov.9th**

## Practice 7 - Improving BTS
You are required to complete the function `improving_bts()`.
For `improving_bts()`, you can use one or more of the following strategies: Minimum Remaining Value, Least constraining value, and Forward checking. You should have a good performance (within 2s) **when N = 16 without GUI**. 

📌 **DDL: Nov.16th**

## Evaluation Criteria

Your submission will be assessed on:

* Clear understanding of every line of your code (no blind copy-paste).
* Successful compilation of the program.
* Correctness of the program logic.
* Reasonable efficiency of the solution.

### Grading Policy

* **On-time submission:** 60–100 points
* **Late submission:** 0 points


In [188]:
import numpy as np
import time

def my_range(start, end):
    if start <= end:
        return range(start, end + 1)
    else:
        return range(start, end - 1, -1)


class Problem:
    char_mapping = ('·', 'Q')

    def __init__(self, n=4):
        self.N = n

    def is_valid(self, state):
        """
        check whether the state satisfies the condition or not.
        : param state: list of points (in (row id, col id) tuple form)
        : return: bool value of valid or not
        """
        board = self.get_board(state)
        res = True
        for point in state:
            i, j = point
            condition1 = board[:, j].sum() <= 1
            condition2 = board[i, :].sum() <= 1
            condition3 = self.pos_slant_condition(board, point)
            condition4 = self.neg_slant_condition(board, point)
            res = res and condition1 and condition2 and condition3 and condition4
            if not res:
                break
        return res

    def is_satisfy(self, state):
        return self.is_valid(state) and len(state) == self.N

    def pos_slant_condition(self, board, point):
        i, j = point
        tmp = min(self.N - i - 1, j)
        start = (i + tmp, j - tmp)
        tmp = min(i, self.N - j - 1)
        end = (i - tmp,  j + tmp)
        rows = my_range(start[0], end[0])
        cols = my_range(start[1], end[1])
        return board[rows, cols].sum() <= 1

    def neg_slant_condition(self, board, point):
        i, j = point
        tmp = min(i, j)
        start = (i - tmp, j - tmp)
        tmp = min(self.N - i - 1, self.N - j - 1)
        end = (i + tmp, j + tmp)
        rows = my_range(start[0], end[0])
        cols = my_range(start[1], end[1])
        return board[rows, cols].sum() <= 1

    def get_board(self, state):
        board = np.zeros([self.N, self.N], dtype=int)
        for point in state:
            board[point] = 1
        return board

    def print_state(self, state):
        board = self.get_board(state)
        print('_' * (2 * self.N + 1))
        for row in board:
            for item in row:
                print(f'|{Problem.char_mapping[item]}', end='')
            print('|')
        print('-' * (2 * self.N + 1))

In [189]:
class Solver:
    def __init__(self, n, method):
        self.N = n
        self.problem = Problem(n)
        self.method = method

    def solve(self, render=True):
        if render:
            import pygame
            w, h = 90 * self.N + 10, 90 * self.N + 10
            screen = pygame.display.set_mode((w, h))
            screen.fill('white')
            action_generator = self.method(self.problem)
            clk = pygame.time.Clock()
            queen_img = pygame.image.load('./queen.png')
            while True:
                for event in pygame.event.get():
                    if event == pygame.QUIT:
                        exit()
                try:
                    actions = next(action_generator)
                    screen.fill('white')
                    for i in range(self.N + 1):
                        pygame.draw.rect(screen, 'black', (i * 90, 0, 10, h))
                        pygame.draw.rect(screen, 'black', (0, i * 90, w, 10))
                    for action in actions:
                        i, j = action
                        screen.blit(queen_img, (10 + 90 * j, 10 + 90 * i))
                    pygame.display.flip()
                except StopIteration:
                    pass
                clk.tick(5)
            pass
        else:
            start_time = time.time()
            actions = None
            for actions in self.method(self.problem):
                pass
            if actions is not None:
                self.problem.print_state(actions)
                print(f"Solution found in {time.time() - start_time:.6f} seconds")
            else:
                print(f"No solution found in {time.time() - start_time:.6f} seconds")

# BTS: Backtracking search

In [190]:
def bts(problem):
    action_stack = []
    next_col = [0] * problem.N
    row = 0
    
    while not problem.is_satisfy(action_stack):
        if row < 0:
            return
        
        if next_col[row] >= problem.N:
            next_col[row] = 0
            row -= 1
            if row >= 0:
                action_stack.pop()
                next_col[row] += 1
            continue
        
        col = next_col[row]
        action_stack.append((row, col))
        
        if problem.is_valid(action_stack):
            row += 1
        else:
            action_stack.pop()
            next_col[row] += 1
            
    yield action_stack
    return

# Improving BTS 
* Which variable should be assigned next?
* In what order should its values be tried?
* Can we detect inevitable failure early?

## Minimum Remaining Value
Choose the variable with the fewest legal values in its domain
## Least constraining value
Given a variable, choose the least constraining value: the one that rules out the fewest values in the remaining variables
## Forward checking
Keep track of remaining legal values for the unassigned variables. Terminate when any variable has no legal values.

In [191]:
def improving_bts(problem):
    action_stack = []  # 存储当前解的皇后位置
    next_col = [0] * problem.N  # 记录每一行下次尝试的列位置
    row = 0  # 当前处理的行
    
    def get_legal_columns(row_idx, current_state):
        # 获取指定行所有合法的列位置
        legal = []
        for col in range(problem.N):
            test_state = current_state + [(row_idx, col)]
            if problem.is_valid(test_state):
                legal.append(col)
        return legal
    
    def count_constraints(row_idx, col, current_state):
        # 计算该选择会约束多少未来的选项
        test_state = current_state + [(row_idx, col)]
        if not problem.is_valid(test_state):
            return float('inf')  # 如果当前选择无效，返回无穷大
        
        constraints = 0
        # 计算该选择对所有未来行的约束数量
        for future_row in range(row_idx + 1, problem.N):
            for future_col in range(problem.N):
                future_test = test_state + [(future_row, future_col)]
                if not problem.is_valid(future_test):
                    constraints += 1
        return constraints
    
    def forward_check(current_state):
        # 前向检查：确保所有未来行都有合法位置可选
        current_row = len(current_state)
        for check_row in range(current_row, problem.N):
            if not get_legal_columns(check_row, current_state):
                return False  # 如果某行没有合法选择，提前失败
        return True
    
    # 缓存每行按LCV排序的列选择，避免重复计算
    ordered_cols = [None] * problem.N
    
    while not problem.is_satisfy(action_stack):
        if row < 0:
            return  # 回溯到根节点之前，无解
        
        # 为当前行初始化排序后的列选择
        if ordered_cols[row] is None:
            legal_columns = get_legal_columns(row, action_stack)
            if legal_columns:
                # 按LCV启发式排序：优先选择约束最少的位置
                legal_columns.sort(key=lambda c: count_constraints(row, c, action_stack))
            ordered_cols[row] = legal_columns
        
        legal_columns = ordered_cols[row]
        
        # 当前行的所有选择都已尝试，需要回溯
        if next_col[row] >= len(legal_columns):
            next_col[row] = 0
            ordered_cols[row] = None  # 清除缓存
            row -= 1
            if row >= 0:
                action_stack.pop()
                next_col[row] += 1
                # 清除后续行的缓存，因为状态已改变
                for i in range(row + 1, problem.N):
                    ordered_cols[i] = None
            continue
        
        # 当前行没有合法选择，直接回溯
        if not legal_columns:
            next_col[row] = 0
            ordered_cols[row] = None
            row -= 1
            if row >= 0:
                action_stack.pop()
                next_col[row] += 1
                for i in range(row + 1, problem.N):
                    ordered_cols[i] = None
            continue
        
        # 尝试当前行的下一个列位置
        col = legal_columns[next_col[row]]
        action_stack.append((row, col))
        
        # 检查是否通过前向检查
        if problem.is_valid(action_stack) and forward_check(action_stack):
            row += 1  # 移动到下一行
            # 清除后续行的缓存，因为约束条件已改变
            for i in range(row, problem.N):
                ordered_cols[i] = None
        else:
            # 不满足约束或前向检查失败，尝试该行的下一列
            action_stack.pop()
            next_col[row] += 1
            
    yield action_stack
    return

## test block

In [192]:
Solver(n=6, method=bts).solve(render=False)

_____________
|·|Q|·|·|·|·|
|·|·|·|Q|·|·|
|·|·|·|·|·|Q|
|Q|·|·|·|·|·|
|·|·|Q|·|·|·|
|·|·|·|·|Q|·|
-------------
Solution found in 0.037530 seconds


In [193]:
Solver(n=16, method=improving_bts).solve(render=False)

_________________________________
|Q|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|
|·|·|·|Q|·|·|·|·|·|·|·|·|·|·|·|·|
|·|·|·|·|·|·|Q|·|·|·|·|·|·|·|·|·|
|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|Q|
|·|·|·|·|·|·|·|·|·|·|·|·|Q|·|·|·|
|·|·|·|·|·|·|·|·|·|·|·|·|·|·|Q|·|
|·|·|·|·|·|·|·|·|·|Q|·|·|·|·|·|·|
|·|·|·|·|·|·|·|·|·|·|·|·|·|Q|·|·|
|·|·|·|·|·|Q|·|·|·|·|·|·|·|·|·|·|
|·|·|Q|·|·|·|·|·|·|·|·|·|·|·|·|·|
|·|·|·|·|Q|·|·|·|·|·|·|·|·|·|·|·|
|·|Q|·|·|·|·|·|·|·|·|·|·|·|·|·|·|
|·|·|·|·|·|·|·|·|·|·|·|Q|·|·|·|·|
|·|·|·|·|·|·|·|·|Q|·|·|·|·|·|·|·|
|·|·|·|·|·|·|·|·|·|·|Q|·|·|·|·|·|
|·|·|·|·|·|·|·|Q|·|·|·|·|·|·|·|·|
---------------------------------
Solution found in 58.019925 seconds
